In [1]:
import sys
sys.path.append( '..' )
from utilities.get_data import get_historical_from_path
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [2]:
mypath = "../database/quick_analysis"
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(mypath+"/"+file)
    
print("All data loaded")

All data loaded


In [3]:
df_list["ETH"]

,open,high,low,close,volume
date,,,,,
2022-05-19 19:09:00,2007.43,2008.40,2003.83,2003.84,569.3954
2022-05-19 19:10:00,2003.84,2004.56,2002.00,2003.98,455.2570
2022-05-19 19:11:00,2003.99,2007.37,2003.98,2005.71,290.0938
2022-05-19 19:12:00,2005.72,2008.03,2005.09,2005.99,380.5731
2022-05-19 19:13:00,2005.99,2006.00,2002.04,2002.26,435.6516
...,...,...,...,...,...
2022-05-20 11:44:00,2049.84,2050.58,2049.04,2049.05,201.9585
2022-05-20 11:45:00,2049.05,2050.98,2048.95,2050.84,248.5187
2022-05-20 11:46:00,2050.85,2053.35,2049.25,2052.64,802.4251


In [4]:
def get_analyisis_from_window(window=24):
    metric_list = {}
    for coin in df_list:
        try:
            df = df_list[coin]
            df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
            df["rvolume"] = df["volume"] * df["close"]
            df["olhc"] = (df["open"] + df["low"] + df["high"] + df["close"])/4
            df["atr"] = (ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'])/df["close"])*100
            # df["norm"].dropna(inplace=True) 
            volatility = df["atr"].mean()
            mean_volume = df["rvolume"].mean()
            std = df["olhc"].std()/df["olhc"].mean()
            perf = (df["close"].iloc[-1] - df["close"].iloc[0]) / df["close"].iloc[0]
            return_vs_vol = perf/volatility
            last_volume = df["rvolume"].iloc[-window:].mean()
            volume_evolution = last_volume / mean_volume
            last_volatility = df.iloc[-window:]["olhc"].std() / df.iloc[-window:]["olhc"].mean()
            last_perf = (df.iloc[-1]["close"] - df.iloc[-window]["close"]) / df.iloc[-window]["close"]
            volatility_evolution = last_volatility / volatility
            last_return_vs_vol = last_perf / last_volatility
            metric_list[coin] = {
                "mean_volume": round(mean_volume), 
                "last_volume": round(last_volume), 
                "volume_evolution": round(volume_evolution,2),
                "mean_volatility": round(volatility,2), 
                "last_volatility": round(last_volatility*100,2),
                "volatility_evolution": round(volatility_evolution*100,2), 
                "return": round(perf*100,2), 
                "last_return":round(last_perf*100,2),
                "return_vs_vol": round(return_vs_vol*100,2),        
                "last_return_vs_vol":round(last_return_vs_vol,2)
                }
        except:
            # print("Error on", coin)
            # raise 
            pass

    return pd.DataFrame.from_dict(metric_list, orient='index')
    

In [5]:
df_metric = get_analyisis_from_window(12).sort_values(by="last_return", ascending=False)
df_metric.iloc[:30]

C:\Users\ayoub\AppData\Local\Temp/ipykernel_17836/3950879056.py:21: RuntimeWarning: invalid value encountered in double_scalars
  last_return_vs_vol = last_perf / last_volatility


,mean_volume,last_volume,volume_evolution,mean_volatility,last_volatility,volatility_evolution,return,last_return,return_vs_vol,last_return_vs_vol
SANTOS,19441,21876,1.13,0.75,1.00,1.33,25.65,2.66,34.26,2.67
TROY,1471,5102,3.47,0.26,0.87,3.31,9.46,2.54,35.82,2.91
FTM,45201,194710,4.31,0.27,0.57,2.11,7.16,2.04,26.24,3.56
MASK,6988,7230,1.03,0.36,0.68,1.89,12.89,2.00,36.01,2.95
KP3R,30486,19983,0.66,1.04,0.91,0.87,17.16,1.97,16.47,2.16
CITY,4665,3567,0.76,0.45,0.59,1.31,12.86,1.63,28.34,2.75
WAXP,1117,5187,4.64,0.11,0.64,5.76,3.96,1.61,35.50,2.51
POWR,1037,5381,5.19,0.12,0.42,3.46,4.95,1.53,40.64,3.63
MTL,3662,8883,2.43,0.24,0.57,2.40,9.50,1.35,40.01,2.36
ONE,8800,11921,1.35,0.20,0.31,1.53,6.45,1.19,31.68,3.84
